### libarary

In [1]:
import pandas as pd
import numpy as np
import apiclient
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

### Global Apisettings

In [2]:
json_path = 'client_secrets.json'

### Function

#### Google Search Console

##### Data discrepancies
You might see a few kinds of data discrepancies in Search Console.
Discrepancies between chart totals and table totals.
You can see differences between the chart totals and the table totals for several reasons:

Chart totals higher:

* The table can show a maximum of 1,000 rows, so some rows might be omitted.
* When filtering by page or query, the "matches" and "does not match" totals might not add up to the unfiltered total. For example, adding the totals for "Queries containing:mouse" and "Queries not containing:mouse" might not equal the total values when no query filters are applied. This is because anonymized queries are omitted, and data is truncated due to serving limitations.
* When viewing by query, anonymized (rare) results are omitted from the table.

Table totals higher:

* When the table is grouped by page or search appearance, the table totals are grouped by URL but the chart totals are still grouped by property. Therefore, if a single property appears multiple times in a single search, it count as 1 result in the chart but multiple results in the table.
* Some search appearances are subcategories of others. For example, Job listing is a subcategory of Rich result, so the same result will be listed in both rows.

In [3]:
def gsc(url, d_list, start_date, end_date) :
    scope=['https://www.googleapis.com/auth/webmasters.readonly']
    credentials=ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
    webmasters=apiclient.discovery.build('webmasters', 'v3', credentials=credentials)
    df = pd.DataFrame(columns = d_list)
    n = 0
    while True:
        row_limit = 25000
        start_Row = n * row_limit
        body = {
            'startDate': start_date,
            'endDate': end_date,
            'dimensions': d_list,
            'rowLimit': row_limit,
            'startRow': start_Row
        }
        res = webmasters.searchanalytics().query(siteUrl=url, body=body).execute()
        if len(res) == 1: break  
        temp_df = pd.io.json.json_normalize(res['rows'])
        for i,d in enumerate(d_list):
            temp_df[d]=temp_df['keys'].apply(lambda x:x[i])
        temp_df.drop(columns='keys',inplace=True)
        df = df.append(temp_df)
        n += 1
    return df

#### Execution sample

**GSC**

##### GSC parameter

In [ ]:
url = '' # fill your domain name
d_list = ['page','date']
start_date = '2019-08-01'# start fill date
end_date = '2019-08-01'# end fill date

In [ ]:
gs_data = gsc(url, d_list, start_date, end_date)

In [ ]:
gs_data.head()